In [ ]:
library(igraph)
# install.packages("ggplot2")
library(ggplot2)
g <- read.graph('facebook_combined.txt', format = 'edgelist', directed = FALSE)

## Problem 17

In [ ]:
CommonNeighbors = function (i, j){
    common_neighbors = length(intersect(i, j))
    return(common_neighbors)
}
Jaccard = function (i, j) {
    jaccard = length(intersect(i, j)) / length(union(i, j))
    return(jaccard)
}
AdamicAdar = function (g, i, j){
    adamic_adar = 0
    for (k in intersect(i, j)){
        adamic_adar = adamic_adar + 1/log10(length(neighbors(g, k)))
    }
    return(adamic_adar)
}

set.seed(1)
Accu_comm = 0
Accu_jacc = 0
Accu_adam = 0
iteration = 10

for (i in Nr){
    neighbors_i = as.numeric(neighbors(network_415, i))
#     print('###########')
#     print(i)
#     print(neighbors_i)
    
    accu_comm = 0
    accu_jacc = 0
    accu_adam = 0
    for (step in 1:iteration) {
        delete_flags = sample(c(0,1), length(neighbors_i), prob=c(0.25,0.75), replace=TRUE)
        tmp = 0
        R_i = seq_len(0)
        for (flag in delete_flags){
            tmp = tmp + 1
            if (flag == 0){
                network_415[i, neighbors_i[tmp]] = 0
                R_i = c(R_i, neighbors_i[tmp])
            }
        }   
        S_i = neighbors(network_415, i)
        
        commons = seq_len(0)
        jaccards = seq_len(0)
        adamicadars = seq_len(0)
        for (j in V(network_415)){
            S_j = neighbors(network_415, j)
            commons = c(commons, CommonNeighbors(S_i, S_j))
            jaccards = c(jaccards, Jaccard(S_i, S_j))
            adamicadars = c(adamicadars, AdamicAdar(network_415, S_i, S_j))
        }
        
        commons = order(commons, decreasing=TRUE)
        jaccards = order(jaccards, decreasing=TRUE)
        adamicadars = order(adamicadars, decreasing=TRUE)
        
#         print(commons)
#         print(jaccards)
#         print(adamicadars)

        P_i_comm = seq_len(0)
        P_i_jacc = seq_len(0)
        P_i_adam = seq_len(0)
        tmp = 0
        while (length(P_i_comm) < length(R_i)){
            tmp = tmp + 1
            if (commons[tmp] != i){
                P_i_comm = c(P_i_comm, commons[tmp])
            }
        }
        while (length(P_i_jacc) < length(R_i)){
            tmp = tmp + 1
            if (jaccards[tmp] != i){
                P_i_jacc = c(P_i_jacc, jaccards[tmp])
            }
        }
        while (length(P_i_adam) < length(R_i)){
            tmp = tmp + 1
            if (adamicadars[tmp] != i){
                P_i_adam = c(P_i_adam, adamicadars[tmp])
            }
        }
#         print(P_i_comm)
#         print(P_i_jacc)
#         print(P_i_adam)
        accu_comm = accu_comm + length(intersect(R_i, P_i_comm)) / length(R_i)
        accu_jacc = accu_jacc + length(intersect(R_i, P_i_jacc)) / length(R_i)
        accu_adam = accu_adam + length(intersect(R_i, P_i_adam)) / length(R_i)
        
        # recover
        tmp = 0
        for (flag in delete_flags){
            tmp = tmp + 1
            if (flag == 0){
                network_415[i, neighbors_i[tmp]] = 1
            }
        }
        
    }
    accu_comm = accu_comm / iteration
    accu_jacc = accu_jacc / iteration
    accu_adam = accu_adam / iteration
    
    Accu_comm = Accu_comm + accu_comm
    Accu_jacc = Accu_jacc + accu_jacc
    Accu_adam = Accu_adam + accu_adam
}
Accu_comm = Accu_comm / length(Nr)
Accu_jacc = Accu_jacc / length(Nr)
Accu_adam = Accu_adam / length(Nr)

print(Accu_comm)
print(Accu_jacc)
print(Accu_adam)